In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()


In [2]:
# {
#     "name": "Maicon",
#     "surName": "Moreira",
#     "age": 23,
#     "contacts": {
#         "email": "maicon.moreira@gmail.com",
#         "phone": "4798769876"
#     }
# }

schema = StructType(
    [
        StructField("name", StringType(), nullable=False),
        StructField("surName", StringType(), nullable=False),
        StructField("age", IntegerType(), nullable=False),
        StructField(
            "contacts",
            StructType(
                [
                    StructField("email", StringType(), nullable=False),
                    StructField("phone", StringType(), nullable=False),
                ]
            ),
            nullable=False,
        ),
    ]
)


In [3]:
df = spark.read.json("sample.json", schema, multiLine=True)

df = df.withColumn("email", df.contacts.email).withColumn("phone", df.contacts.phone).drop("contacts")

In [4]:
df.show()

+------+-------+---+--------------------+----------+
|  name|surName|age|               email|     phone|
+------+-------+---+--------------------+----------+
|Maicon|Moreira| 23|maicon.moreira@gm...|4798769876|
|  João|  Silva| 25|joao.silva@gmail.com|4797531987|
| Maria| Santos| 27|maria.santos@gmai...|4798765432|
+------+-------+---+--------------------+----------+



In [6]:
df.toPandas().to_csv("sample.csv", sep=";")


PermissionError: [Errno 13] Permission denied: 'sample.csv'